In [ ]:
import gromacs as gmx
from gromacs import formats as gmf
import numpy as np
import mdtraj as md
import nglview as nv

In [ ]:
dir='/home/meta/ljocha/ASMSA/My_AE/trpcage/ae'
ftrain=dir + '/trpcage_ds.npy'
pdb = dir + '/trpcage_npt400.pdb'
xtc = dir + '/trpcage_ds_fit.xtc'
dec = '/home/meta/ljocha/ASMSA/My_AE/trpcage/dec_out_LS_matrix_pos_32x32.npy'

train = np.load(ftrain)

In [ ]:
train_view = np.reshape(train,(-1,272,3))

In [ ]:
train.shape

In [ ]:
tr = md.load(pdb)

In [ ]:
tr.xyz = train_view

In [ ]:
v=nv.show_mdtraj(tr)
v

In [ ]:
dec_out_all = np.load(dec)
dec_out_all.shape

In [ ]:
def Unstandardize(x):
    x = np.array(x)
    return ((x*7)-1)

In [ ]:
dec_out_all = Unstandardize(dec_out_all)

In [ ]:
dec_out = dec_out_all[6,18]

In [ ]:
diff = train - dec_out
msd = np.sum(diff * diff,axis=1)

In [ ]:
imin = np.argmin(msd)
imin

In [ ]:
tr2 = md.load(pdb)
tr2.xyz = np.reshape(dec_out_all,(-1,272,3))
#tr2.xyz = train_view[9873]
v2 = nv.show_mdtraj(tr2)
#v2.clear()
v2.add_representation("licorice")
v2

In [ ]:
v.frame=int(imin)

In [ ]:
tr3 = md.load(xtc,top=pdb)
tr3.xyz = np.reshape(dec_out_all,(-1,272,3))
tr3.time = tr3.time[0:tr3.xyz.shape[0]]
tr3.unitcell_vectors = tr3.unitcell_vectors[0:tr3.xyz.shape[0]]

tr3.save_xtc('simulated.xtc')

### Identify backbone atoms

In [ ]:
!cp {pdb} reference.pdb

In [ ]:
gmx.select(s='reference.pdb',on='bb.ndx',select='Backbone')

### Compute backbone dihedrals of the simulated trajectory

In [ ]:
with open('bb.ndx') as bb:
    bb.readline()
    l = " ".join(bb.readlines())
    ndx = np.array(l.split(),np.int32)

In [ ]:
with open('angle.ndx','w') as a:
    a.write('[ dihedrals ]\n')
    for i in range(0,len(ndx)-3):
        a.write('  '.join(map(str,ndx[i:i+4])))
        a.write('\n')

In [ ]:
gmx.gangle(f='simulated.xtc',n='angle.ndx',g1='dihedral',group1='dihedrals',oall='dihedrals.xvg')

In [ ]:
dihs = gmf.XVG('dihedrals.xvg').array[1:]

### Generate restrained topology and run minimization

In [ ]:
gmx.pdb2gmx(f='reference.pdb',o='reference.gro',p='reference.top',water='tip3p',ff='amber99')

In [416]:
with open('min.mdp','w') as m:
    m.write('''
integrator  = steep         ; Algorithm (steep = steepest descent minimization)
emtol       = 1000.0        ; Stop minimization when the maximum force < 1000.0 kJ/mol/nm
emstep      = 0.01          ; Minimization step size
nsteps      = 50000         ; Maximum number of (minimization) steps to perform

nstxout                 = 1         
nstvout                 = 0         
nstfout                 = 0         
nstlog                  = 5
nstxout-compressed      = 1

; Parameters describing how to find the neighbors of each atom and how to calculate the interactions
nstlist         = 1         ; Frequency to update the neighbor list and long range forces
cutoff-scheme   = Verlet    ; Buffered neighbor searching
ns_type         = grid      ; Method to determine neighbor list (simple, grid)
coulombtype     = Cut-off   ; Treatment of long range electrostatic interactions
rcoulomb        = 1.0       ; Short-range electrostatic cut-off
rvdw            = 1.0       ; Short-range Van der Waals cut-off
pbc             = xyz       ; Periodic Boundary Conditions in all 3 dimensions

disre           = Simple
define                  = -DPOSRES 
''')

In [417]:
%%time 
n = 123
for n in range(123,124):
    with open('posre.itp','w') as p:
        p.write('[ dihedral_restraints ]\n')
        for i in range(0,len(ndx)-3):
            p.write('  '.join(map(str,ndx[i:i+4])))
            p.write(f' 1 {dihs[i,n]} 0 5000\n')
    
    gtr = tr3[n]
    gtr.save_gro('min.gro')
    
    gmx.grompp(f='min.mdp',c='min.gro',p='reference.top',o='min.tpr',maxwarn=100000)
    
    gmx.mdrun(deffnm='min')

pod/asmsa-gmx-13210-5qp5d condition met
             :-) GROMACS - gmx grompp, 2023.2-plumed_2.10.0_dev (-:

Executable:   /gromacs/AVX2_256_ts/bin/gmx
Data prefix:  /gromacs/AVX2_256_ts
Working dir:  /mnt/ASMSA
Command line:
  gmx grompp -f min.mdp -c min.gro -p reference.top -o min.tpr -maxwarn 100000

Ignoring obsolete mdp entry 'ns_type'

NOTE 1 [file min.mdp]:
  With Verlet lists the optimal nstlist is >= 10, with GPUs >= 20. Note
  that with the Verlet scheme, nstlist has no effect on the accuracy of
  your simulation.

Generating 1-4 interactions: fudge = 0.5

NOTE 2 [file reference.top, line 2668]:
  System has non-zero total charge: -1.000000
  Total charge should normally be an integer. See
  http://www.gromacs.org/Documentation/Floating_Point_Arithmetic
  for discussion on how close it should be to an integer.



atom name 2 in reference.top and min.gro does not match (H1 - H)

atom name 8 in reference.top and min.gro does not match (HB1 - HB3)

atom name 30 in reference.top

Setting the LD random seed to -1611137289

Generated 2145 of the 2145 non-bonded parameter combinations

Generated 2145 of the 2145 1-4 parameter combinations

Excluding 3 bonded neighbours molecule type 'Protein'

atom name 2 in reference.top and min.gro does not match (H1 - H)

atom name 8 in reference.top and min.gro does not match (HB1 - HB3)

atom name 30 in reference.top and min.gro does not match (HB1 - HB3)

atom name 61 in reference.top and min.gro does not match (HB1 - HB3)

atom name 64 in reference.top and min.gro does not match (HG1 - HG3)

atom name 78 in reference.top and min.gro does not match (HB1 - HB3)

atom name 102 in reference.top and min.gro does not match (HB1 - HB3)

atom name 131 in reference.top and min.gro does not match (HB1 - HB3)

atom name 141 in reference.top and min.gro does not match (HA1 - HA3)

atom name 148 in reference.top and min.gro does not match (HA1 - HA3)

atom name 154 in reference.top and min.gro does not match (HD1 - HD3)

atom name 157 i

pod/asmsa-gmx-13210-5qp5d condition met
             :-) GROMACS - gmx mdrun, 2023.2-plumed_2.10.0_dev (-:

Executable:   /gromacs/AVX2_256_ts/bin/gmx
Data prefix:  /gromacs/AVX2_256_ts
Working dir:  /mnt/ASMSA
Command line:
  gmx mdrun -deffnm min


Back Off! I just backed up min.log to ./#min.log.58#
Reading file min.tpr, VERSION 2023.2-plumed_2.10.0_dev (single precision)
Using 1 MPI process

Non-default thread affinity set, disabling internal thread affinity

Using 1 OpenMP thread 


Back Off! I just backed up min.trr to ./#min.trr.58#

Back Off! I just backed up min.edr to ./#min.edr.58#

Steepest Descents:
   Tolerance (Fmax)   =  1.00000e+03
   Number of steps    =        50000


CPU times: user 102 ms, sys: 147 ms, total: 249 ms
Wall time: 9.5 s



writing lowest energy coordinates.

Back Off! I just backed up min.gro to ./#min.gro.59#

Steepest Descents converged to Fmax < 1000 in 4757 steps
Potential Energy  =  5.9947646e+03
Maximum force     =  9.9673242e+02 on atom 117
Norm of force     =  2.4989255e+02

GROMACS reminds you: "I can't relate to you" (Bad Religion)



In [419]:
v=nv.show_file('min.gro')
v.add_representation('licorice')
v

NGLWidget()

In [ ]:
v2=nv.show_mdtraj(tr3[n])
v2.add_representation('licorice')
v2

In [ ]:
gmx.gangle(f='min.gro',n='angle.ndx',g1='dihedral',group1='dihedrals',oall='min.xvg')

In [ ]:
!cat min.xvg

In [ ]:
dihs[:,n]

In [ ]:
gmx.energy(f='min.edr',input=['Potential','Dih.-Rest.'],o='emin.xvg')

In [ ]:
import matplotlib.pyplot as plt
energ = np.loadtxt('emin.xvg',comments=['#','@'])
plt.plot(energ[:,1],label='Dih.-Rest.')
plt.plot(energ[:,2],label='Potential')
plt.plot(energ[:,2]-energ[:,1],label='Net')
plt.yscale('log')
plt.legend()
plt.show()


In [420]:
mtr = md.load('min.trr',top=pdb)
mtr.xyz = mtr.xyz[-1:0:-1]
v = nv.show_mdtraj(mtr)
v.add_representation('licorice')
v

NGLWidget(max_frame=3762)

In [412]:
mtr.xyz.shape

(1, 272, 3)